### Goal of video
* Give an organized framework for on bar algorithms which we can generalize and make more sophisticated later.
    * An abstract class **OnBarAlgo** that we will implement the subclass **SimpleMLBuySellAlgo**
* We will focus on just 1 onBar algo! In future videos will show you good ways to organize it to many.
* **NOTE**: eventually this approach will generalize to many **OnBarAlgo**s together and we should implement some sort of **BarAlgorithmController** which will handle running a given list/set/dict of these **OnBarAlgo** implementations, but will will save that for another video
* Illustrate simple example of buying stock with a bracket order where we don't adjust price targets at all and only
* Keep it very simple.
  - No order cancelation
  - No order modification
  - No double downing
  - etc..
  - **NOTE** I always adjust price targets, double down, etc..., but want to keep this video understandable

### Remove some annoying warnings!

In [1]:
import warnings
import pandas as pd

# Ignore all FutureWarnings (often from pandas)
warnings.simplefilter(action='ignore', category=FutureWarning)

# Ignore pandas SettingWithCopyWarning specifically
warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)

import warnings

# Ignore the specific warning about timezone information loss
warnings.filterwarnings("ignore", message="Converting to PeriodArray/Index representation will drop timezone information")


### Connect to IB

In [3]:
from ib_async import * 
ib = IB()
util.startLoop()
ib.connect(port=4003,clientId=0)

<IB connected to 127.0.0.1:4003 clientId=0>

### Big Picture Approach with OnBar Long Algorithms:
1. Preface: Getting Bar Data: Historical Bars + Real Time Bars
    - An alternative much easier, but less robust to internet disconnections is to just use historicalbars with keepUptoDate= True for most current one.
3. On Bar Update Callback **onBarUpdate** where you implement logic when new bar arrives

#### Preface: Getting Bar Data: Historical Bars + Real Time Bars
* Get historical data for enough of a period to calculate any features that need it...
    - for example if your bars are 5 minutes and you have RSI 780 as a feature you would need 10 days of historical data since 78 5-min bars in a day 

*  **NOTE 1** I usually have a **Data** class to abstract this away, but we won't do this in this video.
* **NOTE 2** Be careful your logic doesn't result in missing or duplicating some bars
    * For example, if running during market hours then run get live bars before historical bars before the live bars since otherwise you will have a gap.
    

* Define Stock contract we wish to trade

In [4]:
stock = Stock(symbol='AMD',exchange='SMART',currency='USD')
ib.qualifyContracts(stock)

[Stock(conId=4391, symbol='AMD', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='AMD', tradingClass='NMS')]

* Get Realtime 5 second bars

In [5]:
real_time_bars  = ib.reqRealTimeBars(contract=stock,barSize=5,whatToShow='TRADES',useRTH=False)

In [9]:
util.df(real_time_bars)

,time,endTime,open_,high,low,close,volume,wap,count
0,2024-10-31 10:53:35+00:00,-1,146.95,146.95,146.95,146.95,0.0,146.95,0
1,2024-10-31 10:53:40+00:00,-1,146.90,146.90,146.90,146.90,100.0,146.90,1
2,2024-10-31 10:53:45+00:00,-1,146.90,146.90,146.90,146.90,0.0,146.90,0


* Get historical 5 seconds bars
    * Need to loop a few times to get more history due to timeouts

In [10]:
def get_historical_data(contract,num_periods=11):
      barsList = []
      dt = ""
      for i in range(num_periods):
          print(f"Downloading Period {i}")
          bars = ib.reqHistoricalData(
              contract,
              endDateTime=dt,
              durationStr='3 D',
              barSizeSetting='5 secs',
              whatToShow='TRADES', keepUpToDate=False,
              useRTH=True,
              formatDate=1)
          if not bars:
              break
          barsList.append(bars)
          dt = bars[0].date
 
      bars = [b for bars in reversed(barsList) for b in bars]

      historical_df = util.df(bars).set_index('date').copy(deep=True)
      df_cols = ['open', 'high', 'low', 'close', 'volume', 'average', 'dollar_value', 'day_id']
      historical_df['dollar_value'] = historical_df['volume'] * historical_df['average']
      historical_df['day_id'] = historical_df.index.date
      historical_df = historical_df[df_cols]
      return historical_df
historical_df = get_historical_data(stock,11)

In [15]:
historical_df.shape,util.df(real_time_bars).shape

((154440, 8), (92, 9))

#### Bar Upate Callback: **onBarUpdate** (when new bar arrives):
    
1.  Update Dollar Bars, Volume Bars, Time Bars, whatever you Use
2.  Extract Features
3.  Make a Prediction of some sort

4.  Get Action (meaning Run an algorithm) **NOTE** we can have multiple algos and so multiple *get_action* methods
    * For this video's example,
    * If Prediction>=BUY_PROB_THRESHOLD and Close<MAX_BUY_PRICE and NO_ACTIVE_TRADES:
         * Submit bracket buy order for stock
      

### Assumptions: 

#### We have following functions implemented:
1. Get updated aggregate bars: **update_bars**: Black Box function to compute some sort of aggregate bars every 5 seconds
    * For example if using 1 minute bars
        * If we just got 12:30:15 to 12:30:20 then the last 2 bars would be 12:28-20-10:29:20 and 12:29:20-12:30-20
        * If we just got 12:30:20 to 12:30:25 then the last 2 bars would be 12:28-20-10:29:25 and 12:29:20-12:30-25
**NOTE 1** I use some version of dollar bars.

**NOTE 2** If your logic only works for end of whole bars or when some filtering condition is true then you can also return an *IS_COMPLETED* flag for if the bar is completed or not.

2. **extract_features**: From bars to get feature vectors for ML model + other info like Take Profit and Stop Loss Targets! 
4. **Fitted ML model** with model.predict, model.predict_proba or similar

In [16]:
from on_bar_utils import update_bars, extract_features,fitted_ml_model,DOLLAR_BAR_THRESHOLD

* Get new aggregate bars

In [18]:
update_bars(historical_bars_df=historical_df,live_bars=real_time_bars,dollar_bar_threshold=DOLLAR_BAR_THRESHOLD).shape

(1079, 17)

In [19]:
new_agg_bars_df=update_bars(historical_bars_df=historical_df,live_bars=real_time_bars,dollar_bar_threshold=DOLLAR_BAR_THRESHOLD)

* Extract ML Features

In [21]:
features_df = extract_features(new_agg_bars_df)

In [55]:
features_df.columns.tolist()

['RSI_14',
 'RSI_840',
 'RSI_70',
 'PPO_12_26_9',
 'PPOh_12_26_9',
 'PPOs_12_26_9',
 'PPO_40_200_12',
 'PPOh_40_200_12',
 'PPOs_40_200_12',
 'LOGRET_840',
 'LOGRET_1',
 'LOGRET_2',
 'LOGRET_3',
 'AROOND_14',
 'AROONU_14',
 'AROONOSC_14',
 'BOP',
 'MFI_14',
 'serial_correlation_50_1',
 'STOCHRSIk_14_14_3_3',
 'STOCHRSId_14_14_3_3',
 'SKEW_30',
 'SKEW_200',
 'SKEW_90',
 'close',
 'take_profit_target',
 'stop_loss_target']

* Use fitted model to make prediction

In [28]:
fitted_ml_model.predict_proba(features_df.iloc[-1,:])[1]

0.4684235349290563

### Main Algo Loop in **onBarUpdate** callback
* This is what we need to implement
* Note *hasNewBar* isn't so really needed when using real time bars, but let's use it anyways.


```
def on_bar_update(bars,hasNewBar): 
    if hasNewBar:
        # Get Bars in My case Dollar Bars
        df = update_bars(historical_df,real_time_bars, DOLLAR_BAR_THRESHOLD)
        # Extract Features
        df_features = extract_features(df)
        ## Make a Prediction
        probs_cat = fitted_ml_model.predict_proba(df_features.tail(1))[0, 1]
        ## Call on bar algorithms' get_action method one at a time
        #### TODO #######
        ###### on_bar_algo.get_action(df_features,probs_cat) ################
```
        

### Note again I usually wrap this stuff in an **OnBarAlgorithmController** class so I can use multiple onBar algos

### Abstract Class for OnBarAlgorithm
* Snippet of my actual code for onBar algorithms
* You need to implement the get_action method which is called 

In [67]:
from abc import ABC, abstractmethod

class OnBarAlgorithm(ABC):

    def __init__(self,ib:IB):
        self.ib=ib
        self.set_config()

    @abstractmethod
    def set_config(self):
        pass
        
    
    @abstractmethod
    def get_action(self, df_features,prob):
        pass


#### Implement **SimpleMLBuySellAlgo** which will subclass **OnBarAlgorithm**

In [142]:
class SimpleMLBuySellAlgo(OnBarAlgorithm):
    ib:IB
    def __init__(self,ib:IB):
        super().__init__(ib)
        self._ACTIVE_TRADE = False
        self._QTY_SOLD = 0
    def set_config(self): # usually take this from some config.ini file with ConfigParser class
        self.QTY = 200 # Qty to trade
        self.MAX_BUY_PRICE = 260 # Max stock price willing to buy at
        self.DISCOUNT_FROM_LIMIT = -0.05  #(if close is 146.9-0.05 then our buy limit would be 146.85)
        self.PROBS_THRESH_FOR_BUYING = 0.451 
        self.ACTION = 'BUY'
        
    def set_contract(self,contract:Contract):
        self.contract = contract
    
    def get_action(self,df_features,prob):
        close = df_features.iloc[-1].close
        if prob>=self.PROBS_THRESH_FOR_BUYING:
            if close<=self.MAX_BUY_PRICE:
                if not self._ACTIVE_TRADE:
                    limit_price = close - self.DISCOUNT_FROM_LIMIT
                    take_profit_price = df_features.iloc[-1].stop_loss_target#take_profit_target
                    stop_loss_price = df_features.iloc[-1].stop_loss_target-1
                    
                    # Place new bracket order and it will set self._ACTIVE_TRADE = True
                    self.place_bracket_order(limit_price,take_profit_price,stop_loss_price)
    
    def place_bracket_order(self,limit_price,take_profit_price,stop_loss_price):
        bracket_order = ib.bracketOrder(action=self.ACTION,quantity=self.QTY,limitPrice=limit_price,
                                        takeProfitPrice=take_profit_price,stopLossPrice=stop_loss_price,outsideRth=True)
        parent_trade = self.ib.placeOrder(self.contract,bracket_order.parent)
        take_profit_trade = self.ib.placeOrder(self.contract,bracket_order.takeProfit)
        stop_trade = self.ib.placeOrder(self.contract,bracket_order.stopLoss)
        self._ACTIVE_TRADE=True

    def onExecDetails(self,trade:Trade,fill:Fill):
        side = fill.execution.side
        shares = fill.execution.shares

        print(f"New {side} execution for {shares} shares")

        if side == 'SLD':
            self._QTY_SOLD+=shares
            if self._QTY_SOLD == self.QTY:
                self._QTY_SOLD=0
                self._ACTIVE_TRADE=False

    def onNewOrder(self,trade:Trade):
        print("New Order")
        print(trade.order)

    def attach_callbacks(self):
        self.ib.newOrderEvent.clear()
        self.ib.execDetailsEvent.clear()

        self.ib.newOrderEvent+=self.onNewOrder
        self.ib.execDetailsEvent+=self.onExecDetails

        

        
    

                    

### Create an instance of SimpleMLBuySellAlgo and set its params

In [143]:
simple_ml_algo = SimpleMLBuySellAlgo(ib)
simple_ml_algo.set_contract(stock)
simple_ml_algo.attach_callbacks()

### Define the  **on_bar_update** callback

In [144]:
def on_bar_update(bars,hasNewBar): 
    if hasNewBar:
        # Get Bars in My case Dollar Bars
        df = update_bars(historical_df,real_time_bars, DOLLAR_BAR_THRESHOLD)
        # Extract Features
        df_features = extract_features(df)
        ## Make a Prediction
        probs_cat = fitted_ml_model.predict_proba(df_features.tail(1))[0, 1]
        print(f"Prediction prob {probs_cat}")
        ## Call on bar algorithms' get_action method one at a time
        simple_ml_algo.get_action(df_features,probs_cat)


### Start the onBar algo
* First clear the callback to make sure you have nothing else running

In [147]:
real_time_bars.updateEvent.clear()

* Attach callback

In [146]:
real_time_bars.updateEvent+=on_bar_update

Prediction prob 0.469007633623274
New Order
LimitOrder(orderId=2494, action='BUY', totalQuantity=200, lmtPrice=148.06, transmit=False, outsideRth=True)
New Order
LimitOrder(orderId=2495, action='SELL', totalQuantity=200, lmtPrice=147.91, transmit=False, parentId=2494, outsideRth=True)
New Order
StopOrder(orderId=2496, action='SELL', totalQuantity=200, auxPrice=146.91, parentId=2494, outsideRth=True)
New BOT execution for 200.0 shares
New SLD execution for 200.0 shares
Prediction prob 0.469007633623274
New Order
LimitOrder(orderId=2497, action='BUY', totalQuantity=200, lmtPrice=148.06, transmit=False, outsideRth=True)
New Order
LimitOrder(orderId=2498, action='SELL', totalQuantity=200, lmtPrice=147.91, transmit=False, parentId=2497, outsideRth=True)
New Order
StopOrder(orderId=2499, action='SELL', totalQuantity=200, auxPrice=146.91, parentId=2497, outsideRth=True)
New BOT execution for 200.0 shares
New SLD execution for 200.0 shares
Prediction prob 0.4705281245595064
New Order
LimitOrde

### Cancel all open orders

In [141]:
for order in ib.openOrders():
    ib.cancelOrder(order)

In [148]:
util.df(util.df(ib.fills()).execution.tolist())

,execId,time,acctNumber,exchange,side,shares,price,permId,clientId,orderId,liquidation,cumQty,avgPrice,orderRef,evRule,evMultiplier,modelCode,lastLiquidity,pendingPriceRevision
0,0000e0d5.672306ea.01.01,2024-10-31 11:38:19+00:00,DU5269954,ISLAND,BOT,100.0,147.83,1074570277,0,2485,0,100.0,147.830,,,0.0,,1,False
1,0000e0d5.672306eb.01.01,2024-10-31 11:38:55+00:00,DU5269954,ARCA,BOT,100.0,147.80,1074570277,0,2485,0,200.0,147.815,,,0.0,,2,False
2,0000e0d5.672306f9.01.01,2024-10-31 11:41:42+00:00,DU5269954,BATS,BOT,100.0,147.85,1074570280,0,2488,0,100.0,147.850,,,0.0,,2,False
3,0000e0d5.67230799.01.01,2024-10-31 11:44:12+00:00,DU5269954,DRCTEDGE,BOT,200.0,148.05,1074570286,0,2494,0,200.0,148.050,,,0.0,,2,False
4,0000e0d5.6723079a.01.01,2024-10-31 11:44:12+00:00,DU5269954,DRCTEDGE,SLD,200.0,147.96,1074570287,0,2495,0,200.0,147.960,,,0.0,,2,False
5,0000e0d5.672307a0.01.01,2024-10-31 11:44:17+00:00,DU5269954,DRCTEDGE,BOT,200.0,148.05,1074570289,0,2497,0,200.0,148.050,,,0.0,,2,False
6,0000e0d5.672307a1.01.01,2024-10-31 11:44:17+00:00,DU5269954,ISLAND,SLD,200.0,148.01,1074570290,0,2498,0,200.0,148.010,,,0.0,,2,False
7,0000e0d5.672307bc.01.01,2024-10-31 11:44:22+00:00,DU5269954,DRCTEDGE,BOT,200.0,148.09,1074570292,0,2500,0,200.0,148.090,,,0.0,,2,False
8,0000e0d5.672307bd.01.01,2024-10-31 11:44:22+00:00,DU5269954,ISLAND,SLD,200.0,148.05,1074570293,0,2501,0,200.0,148.050,,,0.0,,2,False
9,0000e0d5.672307e6.01.01,2024-10-31 11:44:27+00:00,DU5269954,DRCTEDGE,BOT,200.0,148.09,1074570295,0,2503,0,200.0,148.090,,,0.0,,2,False


In [122]:
real_time_bars[-1]

Prediction prob 0.4619344919625329


RealTimeBar(time=datetime.datetime(2024, 10, 31, 11, 40, 20, tzinfo=datetime.timezone.utc), endTime=-1, open_=147.71, high=147.71, low=147.71, close=147.71, volume=0.0, wap=147.71, count=0)

Prediction prob 0.4619344919625329
Prediction prob 0.4619344919625329


In [125]:
ib.openOrders()

[]